# Peter Norvig's Spell Checker

I came across Peter Norvig’s blog : How to Write a Spelling Corrector. In this post, I have redone the code with minor changes to understand for my own knowledge. 

Let's get started!

We can estimate the probability of a word, P(word), by counting the number of times each word appears in a text file of about a million words, big.txt. 

Big.txt is a concatenation of public domain book excerpts from Project Gutenberg and lists of most frequent words from Wiktionary and the British National Corpus. 

In [ ]:
import re
from collections import Counter


#The function words breaks text into words
def words(text): return re.findall(r'\w+', text.lower())

# Variable WORDS holds a Count of how often each word appears in the text file
WORDS = Counter(words(open('big.txt').read()))

# P estimates the probability of each word, based on the Counter:
def P(word, N=sum(WORDS.values())): 
    return WORDS[word] / N


# Get the most probable spelling correction for the word:
def correction(word): 
    return max(candidates(word), key=P)


# Generate possible spelling corrections for the word:
def candidates(word): 
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])


# We restrict ourselves to words that are known—that is, in the dictionary for a smaller set to work with.
# The subset of words that appear in the dictionary of WORDS:
def known(words): 
    return set(w for w in words if w in WORDS)


#Returns a set of all the edited strings (whether words or not) that can be made with one simple edit:
def edits1(word):
  
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    
    #Split the word in all possible combinations:
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    
    #Remove one character:
    deletes    = [L + R[1:]               for L, R in splits if R]
    
    #Swap two adjacent letters:
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    
    #Change one letter to another:
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    
    #Add a letter:
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)



#Returns a set of all the edited strings (whether words or not) that can be made with two simple edits:  
def edits2(word): 
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [21]:
# Test it out
correction('arrainged')

'arranged'

In [22]:
correction('peotry')

'poetry'

In [23]:
correction('quintessential')

'quintessential'

In [24]:
correction('korrectud')

'corrected'